# SM from IRRmodel into WCM, both with PSO on KGE

The v5 code is a final, cleaned and commented version.

## Temporal resolution of inputs parameters and calibration
The input datasets can have different temporal resolutions:
- hourly
- daily
- multi-daily

Platinum tables of Budrio's SWC, rain and irrigation are hourly.
Climate data such as ET and PET are daily. 
Sigma0 values are daily and multi-daily.
NDVI, LAI and other satellite products are multi-daily.

In order to be self-consistent, the model must be calibrated by taking
into account the temporal scale of variation of quantities.
In particular, the soil water balance (SWB) model requires inputs
that can be hourly or daily. Hourly data have to resampled to be
consistent with daily ones, so SWC, rain and irrigation have to be taken
as their mean and sum values. In this way consistency with ET or PET is
ensured.
Once the SWB model has provided daily values, they can be used as inputs in
the water cloud model, that is calibrated against sigma0 values.

It is clear that an optimal calibration would require an hourly SWB estimate
to match with the correct hours of passage of the satellite. On the other
hand, this would require hourly interpolation of ET and PET data. Moreover, at the
moment the computational power required for such a procedure is not possible
to estimate.

In [8]:
from modules.funcs import *
from modules.funcs_pso import *
from IRRI_WCM.IRRI_WCM_model import *

In [9]:
def pso_calib_irri(PAR):
    """Ausiliary function for PSO optimization"""
    global inputs
    global irri
    n_particles = PAR.shape[0]
    err = np.zeros(n_particles)
    for i in range(n_particles):
        WW,IRR,sig0,KGE = IRR_WCM(PAR[i], inputs, irri)
        err[i] = 1 - KGE
    return err

# Global

In [10]:
filename = f'irr_obs_'

# Input data

Input data formatting convention:
- ausiliary variables for extraction of data (directory name, file name, etc...)
- extraction into pd dataframe
- cleaning, resampling: drop unnecessary columns, set index to daily DateIndex

In [11]:
print('Starting...\n'+
      '#-------------------------------------------------------------\n'+
      'Use of satellite-derived SM is provided for comparison, not calibration.\n')
verbose = True if input("Verbose data extraction? (Describe datasets/files) [y/n]")=='y' else False

Starting...
#-------------------------------------------------------------
Use of satellite-derived SM is provided for comparison, not calibration.



Verbose data extraction? (Describe datasets/files) [y/n] y


In [103]:
#----------------------------------------------------------------------------
# Field data from TEST_SITE
# Daily data from 2015 to 2017, various gaps

# Data extracted:
# - rain (as input SWB)
# - EPOT = potential evapotranspiration (as input SWB)

namesite = 'ITALY_BUDRIO'
siteID = '5'
namefig = namesite+'_'+siteID

sitedf = xr.open_dataset(f'IRRmodel\TEST_SITE\TEST_SITE_{namesite}.nc',
                         engine='netcdf4').to_dataframe();
sitedf = sitedf.rename(columns={'Time_days':'Date'})
sitedf = sitedf.set_index('Date')
sitedf = sitedf.loc[:,[col for col in sitedf.columns if col.endswith(siteID)]]
if verbose: sitedf.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1037 entries, 2015-01-01 to 2017-11-02
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Irrigation_5        214 non-null    float64
 1   Rainfall_5          1037 non-null   float64
 2   ET_5                1037 non-null   float64
 3   PET_5               1037 non-null   float64
 4   SSM_ASCAT_5         1009 non-null   float64
 5   SSM_CCI_combined_5  998 non-null    float64
 6   SSM_CCI_active_5    972 non-null    float64
 7   SSM_CCI_passive_5   925 non-null    float64
 8   SSM_SMAP_5          609 non-null    float64
 9   SSM_SMOS_5          598 non-null    float64
 10  SSM_THEIA_5         179 non-null    float64
 11  SSM_RT1_5           241 non-null    float64
dtypes: float64(12)
memory usage: 105.3 KB


In [232]:
#----------------------------------------------------------------------------
# Sigma0 values
# Daily values of backscattering from 2014 to 2022 (complete S1 series)
# Data extracted:
# - sigma0 values, VV and VH
# - angle of incidence of reference orbit (nearest to 40°)

sigma_df = pd.read_csv('Data\Budrio_half-right.csv', delimiter='\t');
sigma_df['Datetime'] = sigma_df.Date.apply(lambda x : pd.to_datetime(x))
sigma_df.Date = sigma_df.Date.apply(lambda x : pd.to_datetime(x).round(freq='D'))
sigma_df = sigma_df.set_index('Date')
if verbose: sigma_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1150 entries, 2014-10-12 to 2022-11-28
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Angle[°]     1150 non-null   float64       
 1   Geometry     1150 non-null   object        
 2   Orb          1150 non-null   int64         
 3   Pass         1150 non-null   object        
 4   VV_norm[dB]  1150 non-null   float64       
 5   VH_norm[dB]  1150 non-null   float64       
 6   Datetime     1150 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 71.9+ KB


In [233]:
sigma_df.resample('1D')

,Angle[°],Geometry,Orb,Pass,VV_norm[dB],VH_norm[dB],Datetime
Date,,,,,,,
2014-10-12,41.268929,Budrio_half-right,95,DESCENDING,-7.804083,-17.121268,2014-10-12 07:00:00
2014-10-24,41.271503,Budrio_half-right,95,DESCENDING,-10.874000,-21.725451,2014-10-24 07:00:00
2014-11-17,41.107655,Budrio_half-right,95,DESCENDING,-7.428182,-16.462105,2014-11-17 06:00:00
2014-11-29,41.106918,Budrio_half-right,95,DESCENDING,-7.207139,-16.972004,2014-11-29 06:00:00
2014-12-23,41.107285,Budrio_half-right,95,DESCENDING,-9.750607,-18.082929,2014-12-23 06:00:00
...,...,...,...,...,...,...,...
2022-11-13,37.399632,Budrio_half-right,117,ASCENDING,-11.408157,-21.161616,2022-11-12 18:00:00
2022-11-16,31.267229,Budrio_half-right,168,DESCENDING,-8.479531,-17.049766,2022-11-16 06:00:00
2022-11-23,41.307030,Budrio_half-right,95,DESCENDING,-6.117674,-15.749171,2022-11-23 06:00:00


In [68]:
#----------------------------------------------------------------------------
# Budrio field data from Platinum tables
# Hourly field data
# Data extracted:
# - rain (as input SWB)
# - SWC (as comparison)
# - irrigation (as comparison)

platinum = pd.ExcelFile('Data\Platinum_Budrio.xlsx', engine='openpyxl')
platinum = platinum.parse('2017_1h')
platinum['Ora_1'] = pd.to_datetime(platinum['Ora'].astype('str')).apply(lambda x: x.time())
platinum['Data_1'] = pd.to_datetime(platinum['Data'].astype('str')).apply(lambda x: x.date())

# Column 'Date' contains date+hour = hourly information
# Column 'Data' contains only date = daily information
platinum['Date'] = platinum.apply(lambda r : dtt.datetime.combine(r['Data_1'],r['Ora_1']),1)
platinum = platinum.drop(['ID', 'Data_1', 'Ora_1', '214Pb[cps]'],axis=1)
platinum = platinum.set_index('Date')
platinum.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5125 entries, 2017-04-03 11:00:00 to 2017-11-02 23:00:00
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Data             5125 non-null   datetime64[ns]
 1   Ora              5125 non-null   object        
 2   SWC[m3/m3]       4884 non-null   float64       
 3   Pioggia[mm]      5045 non-null   float64       
 4   Irrigazione[mm]  5125 non-null   float64       
 5   Temperatura[°C]  5094 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 280.3+ KB


In [94]:
# Hourly df with all data from Platinum table and passage of satellite at correct hours
df_h = pd.merge(right=platinum, left=sigma_df, on='Date', how='right'); df_h.info()
print('\n#----------------------------------------------------------------------------\n')
# Daily df with all data from Platinum table and passage of satellite
df = pd.merge(right=platinum, left=sigma_df, on='Date', how='inner'); df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5125 entries, 2017-04-03 11:00:00 to 2017-11-02 23:00:00
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Angle[°]         107 non-null    float64       
 1   Geometry         107 non-null    object        
 2   Orb              107 non-null    float64       
 3   Pass             107 non-null    object        
 4   VV_norm[dB]      107 non-null    float64       
 5   VH_norm[dB]      107 non-null    float64       
 6   Data             5125 non-null   datetime64[ns]
 7   Ora              5125 non-null   object        
 8   SWC[m3/m3]       4884 non-null   float64       
 9   Pioggia[mm]      5045 non-null   float64       
 10  Irrigazione[mm]  5125 non-null   float64       
 11  Temperatura[°C]  5094 non-null   float64       
dtypes: datetime64[ns](1), float64(8), object(3)
memory usage: 520.5+ KB

#----------------------------------------

In [46]:
# Dates
D_0 = sitedf.index; set1 = {x for x in D_0}
D_1 = platinum['Data']; set2 = {x for x in D_1}
d = np.sort(np.array([*set1.intersection(set2)])) # dates, full

d_sat = [pd.Timestamp(df.Data[i]) for i in range(len(df.index))]
set1 = {x for x in d}; set2 = {x for x in d_sat}
d_sat = np.sort(np.array([*set1.intersection(set2)])) # dates, passage of sat

In [48]:
# Backscattering
# t_deg is the angle of orbit 95, to which all orbits are normalized
# obs are the values of sigma0

t_deg = [np.mean(df.loc[df.Orb==95]['Angle[°]'].values) for i in range(len(df.Orb.values))]
obs = df.loc[d_sat]['VV_norm[dB]'].values
obs_VH = df.loc[d_sat]['VH_norm[dB]'].values
# rt1 = df.loc[d_sat]['RT1[-]']

In [ ]:
# Vegetation indexes
#############################################################################
# redefine LAI dataset since I have changed the structure of df
# take LAI directly and perform all operations of cleaning directly here
# and merge datasets here

LAI = df.loc[d_sat]['LAI[m2/m2]'].values
cr = (db_lin(obs_VH)/db_lin(obs))

df_NDVI = pd.read_csv(f'Data\\NDVI_GEE.csv', delimiter = "\t")
df_NDVI.Datetime = df_NDVI.Datetime.apply(lambda x:pd.Timestamp(x))
NDVI = df_NDVI.set_index('Datetime').loc[d_sat]['NDVI'].values

In [ ]:
P = platinum.resample('1D',on='Date').sum().loc[d]['Pioggia[mm]'].values
EPOT = sitedf.loc[d][f'PET_{siteID}'].values # potential evapotranspiration (measured)
IRRobs = platinum.resample('1D',on='Date').sum().loc[d]['Irrigazione[mm]'].values
Wobs_gap = platinum.resample('1D',on='Date').mean().loc[d]['SWC[m3/m3]'].values
Wobs = platinum.resample('1D',on='Date').mean().loc[d]['SWC[m3/m3]'].interpolate(method='linear').values
WWobs_gap = (Wobs_gap-np.min(Wobs))/(np.max(Wobs)-np.min(Wobs))
WWobs = norm(Wobs)

index = input('Please provide name of vegetation index to use as input. [Options: LAI, cr, vh, NDVI]')
if index=='LAI': veg=LAI; label_veg='LAI[-]'
elif index=='NDVI': veg=NDVI; label_veg='NDVI[-]'
elif index=='cr': veg=cr; label_veg=r'$\sigma^0$ cross ratio VH/VV [-]'
elif index=='vh': veg=db_lin(obs_VH); label_veg=r'$\sigma^0$ [VH] [dB]'
else: raise NameError('Please select one of the available options')

Starting...
#-------------------------------------------------------------
Use of satellite-derived SM is provided for comparison, not calibration.



Please provide name of vegetation index to use as input. [Options: LAI, cr, vh, NDVI] NDVI


In [245]:
platinum.resample('1D').mean()

,SWC[m3/m3],Pioggia[mm],Irrigazione[mm],Temperatura[°C]
Date,,,,
2017-04-03,0.178090,0.000000,0.0,19.355038
2017-04-04,0.167481,0.000000,0.0,16.422625
2017-04-05,0.162156,0.010417,0.0,13.484700
2017-04-06,0.157279,0.000000,0.0,12.812754
2017-04-07,0.153476,0.000000,0.0,13.099254
...,...,...,...,...
2017-10-29,0.149298,0.000000,0.0,13.998665
2017-10-30,0.147555,0.000000,0.0,12.068414
2017-10-31,0.149344,0.000000,0.0,10.321377


In [258]:
# sitedf, platinum, sigma_df
platinum_reduced=platinum.resample('1D').mean()
#platinum_reduced=platinum_reduced.drop(['Pioggia[mm]','Irrigazione[mm]','Temperatura[°C]'])

daily_field = pd.merge(
    right=sitedf,
    left=platinum_reduced,
    on='Date',
    how='inner',
); daily_field

,SWC[m3/m3],Pioggia[mm],Irrigazione[mm],Temperatura[°C],Irrigation_5,Rainfall_5,ET_5,PET_5,SSM_ASCAT_5,SSM_CCI_combined_5,SSM_CCI_active_5,SSM_CCI_passive_5,SSM_SMAP_5,SSM_SMOS_5,SSM_THEIA_5,SSM_RT1_5
Date,,,,,,,,,,,,,,,,
2017-04-03,0.178090,0.000000,0.0,19.355038,0.0,0.0,1.51705,4.66754,0.08980,0.36576,0.13432,0.29436,NaN,0.08741,NaN,NaN
2017-04-04,0.167481,0.000000,0.0,16.422625,0.0,0.0,1.57788,4.82153,0.06870,NaN,0.01655,0.10420,0.08693,NaN,NaN,0.00018
2017-04-05,0.162156,0.010417,0.0,13.484700,0.0,0.2,1.63870,4.97552,0.00540,0.13537,NaN,0.15840,0.09196,0.17983,0.86884,0.00004
2017-04-06,0.157279,0.000000,0.0,12.812754,0.0,0.0,1.69952,5.12951,0.08510,0.26973,0.04692,0.39945,NaN,0.01891,0.22953,NaN
2017-04-07,0.153476,0.000000,0.0,13.099254,0.0,0.0,1.76034,5.28349,0.04835,0.21491,0.11549,0.20043,0.09628,0.24725,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-10-29,0.149298,0.000000,0.0,13.998665,0.0,0.0,0.74039,2.26284,0.32090,0.64323,0.38495,0.33940,0.06273,NaN,NaN,NaN
2017-10-30,0.147555,0.000000,0.0,12.068414,0.0,0.0,0.74746,2.18480,0.27873,0.55662,0.40493,0.28759,0.01563,0.12746,NaN,NaN
2017-10-31,0.149344,0.000000,0.0,10.321377,0.0,0.0,0.75452,2.10676,0.28023,0.66521,0.28952,0.50954,NaN,NaN,NaN,0.33737


In [241]:
daily = pd.merge(
    right=daily_field,
    left=sigma_df,
    right_on='Date',
    left_on='Date',
    how='right'
)
#daily=daily.set_index('Date')
daily.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 213 entries, 2017-04-04 to 2017-11-02
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Angle[°]            106 non-null    float64       
 1   Geometry            106 non-null    object        
 2   Orb                 106 non-null    float64       
 3   Pass                106 non-null    object        
 4   VV_norm[dB]         106 non-null    float64       
 5   VH_norm[dB]         106 non-null    float64       
 6   Datetime            106 non-null    datetime64[ns]
 7   Data                213 non-null    datetime64[ns]
 8   Ora                 213 non-null    object        
 9   SWC[m3/m3]          204 non-null    float64       
 10  Pioggia[mm]         208 non-null    float64       
 11  Irrigazione[mm]     213 non-null    float64       
 12  Temperatura[°C]     210 non-null    float64       
 13  Irrigation_5        213 non-nul

In [222]:
daily = pd.merge_asof(
    right=daily_field,
    left=sigma_df,
    #right_on='Data', left_on='Date',
    # on='Date',
    left_index=True,
    right_index=True,
    direction='nearest',
    tolerance=pd.Timedelta("4d"),
)
#daily=daily.set_index('Date')
daily.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1150 entries, 2014-10-12 to 2022-11-28
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Angle[°]            1150 non-null   float64       
 1   Geometry            1150 non-null   object        
 2   Orb                 1150 non-null   int64         
 3   Pass                1150 non-null   object        
 4   VV_norm[dB]         1150 non-null   float64       
 5   VH_norm[dB]         1150 non-null   float64       
 6   Data                109 non-null    datetime64[ns]
 7   Ora                 109 non-null    object        
 8   SWC[m3/m3]          105 non-null    float64       
 9   Pioggia[mm]         105 non-null    float64       
 10  Irrigazione[mm]     109 non-null    float64       
 11  Temperatura[°C]     105 non-null    float64       
 12  Irrigation_5        109 non-null    float64       
 13  Rainfall_5          109 non-nu

In [226]:
daily

,Angle[°],Geometry,Orb,Pass,VV_norm[dB],VH_norm[dB],Data,Ora,SWC[m3/m3],Pioggia[mm],...,ET_5,PET_5,SSM_ASCAT_5,SSM_CCI_combined_5,SSM_CCI_active_5,SSM_CCI_passive_5,SSM_SMAP_5,SSM_SMOS_5,SSM_THEIA_5,SSM_RT1_5
Date,,,,,,,,,,,,,,,,,,,,,
2014-10-12,41.268929,Budrio_half-right,95,DESCENDING,-7.804083,-17.121268,NaT,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-10-24,41.271503,Budrio_half-right,95,DESCENDING,-10.874000,-21.725451,NaT,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-11-17,41.107655,Budrio_half-right,95,DESCENDING,-7.428182,-16.462105,NaT,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-11-29,41.106918,Budrio_half-right,95,DESCENDING,-7.207139,-16.972004,NaT,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-12-23,41.107285,Budrio_half-right,95,DESCENDING,-9.750607,-18.082929,NaT,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-13,37.399632,Budrio_half-right,117,ASCENDING,-11.408157,-21.161616,NaT,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-16,31.267229,Budrio_half-right,168,DESCENDING,-8.479531,-17.049766,NaT,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-23,41.307030,Budrio_half-right,95,DESCENDING,-6.117674,-15.749171,NaT,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [230]:
daily.iloc[250:300,:]

,Angle[°],Geometry,Orb,Pass,VV_norm[dB],VH_norm[dB],Data,Ora,SWC[m3/m3],Pioggia[mm],...,ET_5,PET_5,SSM_ASCAT_5,SSM_CCI_combined_5,SSM_CCI_active_5,SSM_CCI_passive_5,SSM_SMAP_5,SSM_SMOS_5,SSM_THEIA_5,SSM_RT1_5
Date,,,,,,,,,,,,,,,,,,,,,
2017-05-28,31.388741,Budrio_half-right,168,DESCENDING,-11.150780,-21.672720,2017-05-28,00:00:00,0.268894,0.0,...,3.75561,7.84133,NaN,0.14028,NaN,0.14025,0.11344,NaN,NaN,0.62220
2017-05-29,41.313030,Budrio_half-right,95,DESCENDING,-11.123042,-21.696000,2017-05-29,00:00:00,0.215570,0.0,...,3.71563,7.75966,0.15190,0.23468,0.12350,NaN,NaN,0.07670,0.89492,NaN
2017-05-31,37.380871,Budrio_half-right,117,ASCENDING,-11.063542,-20.744070,2017-05-31,00:00:00,0.313189,0.0,...,3.63565,7.59631,0.12253,0.30747,0.16866,0.21693,0.14973,0.14722,NaN,NaN
2017-06-03,31.317732,Budrio_half-right,168,DESCENDING,-11.276048,-21.570964,2017-06-03,00:00:00,0.237202,0.0,...,3.49481,7.41910,0.16283,0.36563,0.14840,0.26433,0.00982,0.05050,NaN,0.40480
2017-06-04,41.291061,Budrio_half-right,95,DESCENDING,-13.384052,-20.702730,2017-06-04,00:00:00,0.209401,0.0,...,3.43393,7.40524,0.21325,0.36026,0.19126,0.19928,0.08456,NaN,0.78566,0.32903
2017-06-06,37.538597,Budrio_half-right,117,ASCENDING,-10.755598,-20.232515,2017-06-06,00:00:00,0.173680,0.0,...,3.31219,7.37751,0.19580,0.26660,0.14307,0.18260,NaN,NaN,NaN,NaN
2017-06-09,31.392635,Budrio_half-right,168,DESCENDING,-11.769136,-21.366021,2017-06-09,00:00:00,0.213418,0.0,...,3.12958,7.33592,0.16630,0.45127,0.20943,0.26433,NaN,0.06565,NaN,0.55808
2017-06-10,41.313889,Budrio_half-right,95,DESCENDING,-11.944370,-19.937096,2017-06-10,00:00:00,NaN,0.0,...,3.06871,7.32205,0.18245,0.18252,0.15917,0.08740,0.07840,NaN,0.92205,0.38952
2017-06-12,37.379333,Budrio_half-right,117,ASCENDING,-11.537925,-20.227877,2017-06-12,00:00:00,NaN,0.0,...,3.03752,7.53148,0.16580,0.42877,0.19161,0.26433,0.10837,NaN,NaN,NaN


In [223]:
daily.loc[d]

KeyError: "[Timestamp('2017-04-03 00:00:00'), Timestamp('2017-04-06 00:00:00'), Timestamp('2017-04-08 00:00:00'), Timestamp('2017-04-09 00:00:00'), Timestamp('2017-04-12 00:00:00'), Timestamp('2017-04-14 00:00:00'), Timestamp('2017-04-15 00:00:00'), Timestamp('2017-04-18 00:00:00'), Timestamp('2017-04-20 00:00:00'), Timestamp('2017-04-21 00:00:00'), Timestamp('2017-04-24 00:00:00'), Timestamp('2017-04-26 00:00:00'), Timestamp('2017-04-27 00:00:00'), Timestamp('2017-04-30 00:00:00'), Timestamp('2017-05-02 00:00:00'), Timestamp('2017-05-03 00:00:00'), Timestamp('2017-05-06 00:00:00'), Timestamp('2017-05-08 00:00:00'), Timestamp('2017-05-09 00:00:00'), Timestamp('2017-05-12 00:00:00'), Timestamp('2017-05-14 00:00:00'), Timestamp('2017-05-15 00:00:00'), Timestamp('2017-05-18 00:00:00'), Timestamp('2017-05-20 00:00:00'), Timestamp('2017-05-21 00:00:00'), Timestamp('2017-05-24 00:00:00'), Timestamp('2017-05-26 00:00:00'), Timestamp('2017-05-27 00:00:00'), Timestamp('2017-05-30 00:00:00'), Ti

In [208]:
pd.DatetimeIndex(daily.index.date)

AttributeError: 'DatetimeIndex' object has no attribute 'loc'

In [216]:
d_index = pd.DatetimeIndex(d); d_index

DatetimeIndex(['2017-04-03', '2017-04-04', '2017-04-05', '2017-04-06',
               '2017-04-07', '2017-04-08', '2017-04-09', '2017-04-10',
               '2017-04-11', '2017-04-12',
               ...
               '2017-10-24', '2017-10-25', '2017-10-26', '2017-10-27',
               '2017-10-28', '2017-10-29', '2017-10-30', '2017-10-31',
               '2017-11-01', '2017-11-02'],
              dtype='datetime64[ns]', length=214, freq=None)

In [217]:
d_index[0]

Timestamp('2017-04-03 00:00:00')

In [219]:
daily.loc[d_index[0],:]

KeyError: Timestamp('2017-04-03 00:00:00')

In [213]:
pd.DatetimeIndex(daily.index.values)

DatetimeIndex(['2014-10-12', '2014-10-24', '2014-11-17', '2014-11-29',
               '2014-12-23', '2015-01-04', '2015-01-28', '2015-02-09',
               '2015-02-21', '2015-03-05',
               ...
               '2022-10-23', '2022-10-30', '2022-11-01', '2022-11-04',
               '2022-11-11', '2022-11-13', '2022-11-16', '2022-11-23',
               '2022-11-25', '2022-11-28'],
              dtype='datetime64[ns]', length=1150, freq=None)

In [184]:
sigma_df.index

DatetimeIndex(['2014-10-12', '2014-10-24', '2014-11-17', '2014-11-29',
               '2014-12-23', '2015-01-04', '2015-01-28', '2015-02-09',
               '2015-02-21', '2015-03-05',
               ...
               '2022-10-23', '2022-10-30', '2022-11-01', '2022-11-04',
               '2022-11-11', '2022-11-13', '2022-11-16', '2022-11-23',
               '2022-11-25', '2022-11-28'],
              dtype='datetime64[ns]', name='Date', length=1150, freq=None)

In [182]:
daily.loc[d[0]]

KeyError: Timestamp('2017-04-03 00:00:00')

In [171]:
d_sat[0]

Timestamp('2017-04-04 00:00:00')